# Exploring Land Surface Temperature

[ESA Land Surface Temperature Climate Change Initiative (LST_cci): Monthly Multisensor Infra-Red (IR) Low Earth Orbit (LEO) land surface temperature (LST) time series level 3 supercollated (L3S) global product (1995-2020), version 2.00](https://catalogue.ceda.ac.uk/uuid/785ef9d3965442669bff899540747e28).


In [ ]:
import typing
import warnings

import geopandas as gpd
import numpy as np
import pandas as pd

from shapely.errors import ShapelyDeprecationWarning

warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)


## Obtain Geometries for Cholera Outbreak Regions


First, let's read our outbreak data and select all outbreaks in admin2 regions:


In [ ]:
admin2_outbreaks_df = (
    pd.read_csv("../data/outbreak_data.csv", parse_dates=["start_date", "end_date"])
    .query("spatial_scale == 'admin2'")
    .assign(
        start_year=lambda df: df.start_date.dt.year,
        start_month=lambda df: df.start_date.dt.month,
        duration_in_months=lambda df: np.ceil(
            (df.end_date - df.start_date) / np.timedelta64(1, "M")
        ).astype(int),
    )
)

admin2_outbreaks_df


Merge our distinct location period IDs with the shapefile to obtain the geometries for
only our distinct admin2 outbreak regions:


In [ ]:
geolocations_gdf = typing.cast(
    gpd.GeoDataFrame,
    gpd.read_file("../data/AfricaShapefiles/total_shp_0427.shp")
    .rename(columns={"lctn_pr": "location_period_id"})
    .merge(
        admin2_outbreaks_df[["location_period_id"]].drop_duplicates(),
        how="inner",
        on="location_period_id",
    )
    .drop_duplicates("geometry", keep=False),
)

geolocations_gdf


In [ ]:
display(geolocations_gdf.crs)
geolocations_gdf.boundary.plot(linewidth=0.2)
